In [10]:
from __future__ import division

import json
import pickle
import pandas as pd
import numpy as np

In [22]:


with open('../data/general/matches_2017_2018_v1.json') as matches_json:
    matches_json = pd.read_json(matches_json)
    
matches = [60565]


no_team = 0
home_team = 0
away_team = 0
match_count = 1



data_height = 6000 # number of seconds in per match
data_width = 128 # number of features for fector


for id in matches:
    with open('../data/match_' + str(id) + '/match_data_' + str(id) + '.json', 'r') as file:
        team = json.load(file)
        
    
        
    # defining 3 distinct id in game
    referee_id = 0
    home_team_id = int(team[1]['teamId'])
    away_team_id = int(team[2]['teamId'])
    

    # home and away teams' pos_avrg for 5 min, pos at the time and count of how many time they own the ball
    home_team_pos_avrg = {'xpos' : {}, 'ypos' : {}}
    away_team_pos_avrg = {'xpos' : {}, 'ypos' : {}}
    
    home_team_pos_t = {'xpos' : {}, 'ypos' : {}}
    away_team_pos_t = {'xpos' : {}, 'ypos' : {}}      
    
    home_team_pos_count = {}
    away_team_pos_count = {}
    
    # referee pos at the time
    referee_pos_t = {}
    referee_pos_t['xpos'] = 0
    referee_pos_t['ypos'] = 0
    
    
    # players position ID 
    player_positionId = {'homeTeam' : {}, 'awayTeam' : {}}
    
    
    with open('../data/match_' + str(id) + '/roster_data_' + str(id) + '.json', 'r') as file:
        match_squad = json.load(file)
        
    for roaster in match_squad:
        if roaster['teamId'] == home_team_id:
            home_team_pos_avrg['xpos'][roaster['jerseyNumber']] = 0
            home_team_pos_avrg['ypos'][roaster['jerseyNumber']] = 0
            home_team_pos_count[roaster['jerseyNumber']] = 0
            player_positionId['homeTeam'][roaster['jerseyNumber']] = roaster['positionId'] 
            
        elif roaster['teamId'] == away_team_id:
            away_team_pos_avrg['xpos'][roaster['jerseyNumber']] = 0
            away_team_pos_avrg['ypos'][roaster['jerseyNumber']] = 0
            away_team_pos_count[roaster['jerseyNumber']] = 0
            player_positionId['awayTeam'][roaster['jerseyNumber']] = roaster['positionId']
            
            
    # create array of x input data for every match
    x_intput = np.ndarray(shape=(data_height, data_width), dtype=np.float)        
            
    with open('../data/match_' + str(id) + '/per_sec_data_' + str(id) + '.json', 'r') as file:
        data_persec = json.load(file)
        
        
    # create first average positon data for assigning player position
    for t in data_persec:
        
        i = int(t['minute'])*60 + int(t['second'])
        x_pos = t['xpos']
        y_pos = t['ypos']
        team_id_t = int(t['teamId'])
        jersey_number_t = int(t['jerseyNumber'])
        has_ball_teamId_t = int(t['hasballTeamId'])
        
        if i==300:
            break
        
        if team_id_t == home_team_id and has_ball_teamId_t == away_team_id:
            home_team_pos_avrg['xpos'][jersey_number_t] += x_pos
            home_team_pos_avrg['ypos'][jersey_number_t] += y_pos
            if x_pos!=0 or y_pos!=0:
                home_team_pos_count[jersey_number_t] +=1                

        elif team_id_t == away_team_id and has_ball_teamId_t == home_team_id:
            away_team_pos_avrg['xpos'][jersey_number_t] += x_pos
            away_team_pos_avrg['ypos'][jersey_number_t] += y_pos
            if x_pos!=0 or y_pos!=0:
                away_team_pos_count[jersey_number_t] +=1
            
        
    
           
    tmp_t = 0 # temporary t 
        
    for d in data_persec:
        
        
        i = int(d['minute'])*60 + int(d['second'])
        x_pos = d['xpos']
        y_pos = d['ypos']
        team_id_t = int(d['teamId'])
        jersey_number_t = int(d['jerseyNumber'])
        has_ball_teamId_t = int(d['hasballTeamId'])
        
        
        # assign player to groups then clear pos_avr values in each 5 mins
        if (i+1)%300 == 0 and tmp_t != i+1 and i>299:
            
            tmp_t = i+1
            
            for key, value in home_team_pos_avrg['xpos'].items():
                try:
                    home_team_pos_avrg['xpos'][key] /= home_team_pos_count[key]
                except ZeroDivisionError:
                    home_team_pos_avrg['xpos'][key] = 0
                
            for key, value in home_team_pos_avrg['ypos'].items():
                try:
                    home_team_pos_avrg['ypos'][key] /= home_team_pos_count[key]
                except ZeroDivisionError:
                    home_team_pos_avrg['ypos'][key] = 0
                    
            
            # away team
            for key, value in away_team_pos_avrg['xpos'].items():
                try:
                    away_team_pos_avrg['xpos'][key] /= away_team_pos_count[key]
                except ZeroDivisionError:
                    away_team_pos_avrg['xpos'][key] = 0
                
            for key, value in away_team_pos_avrg['ypos'].items():
                try:
                    away_team_pos_avrg['ypos'][key] /= away_team_pos_count[key]
                except ZeroDivisionError:
                    away_team_pos_avrg['ypos'][key] = 0
                    
                    
            print('\n')
            print(i+1)
            print(i)
            print(home_team_pos_avrg)
            print(away_team_pos_avrg)
            print('\n')
                
        
        
            # home team
            for key, value in home_team_pos_avrg['xpos'].items():
                home_team_pos_avrg['xpos'][key] = 0
                
            for key, value in home_team_pos_avrg['ypos'].items():
                home_team_pos_avrg['ypos'][key] = 0
                          
            for key, value in home_team_pos_count.items():
                home_team_pos_count[key] = 0
                          
            
            # away team
            for key, value in away_team_pos_avrg['xpos'].items():
                away_team_pos_avrg['xpos'][key] = 0
                
            for key, value in away_team_pos_avrg['ypos'].items():
                away_team_pos_avrg['ypos'][key] = 0
                          
            for key, value in away_team_pos_count.items():
                away_team_pos_count[key] = 0
            
        
        
        
        if team_id_t == home_team_id:
            
            if player_positionId['homeTeam'][jersey_number_t] == 1:
                home_team_pos_avrg['xpos'][jersey_number_t] += x_pos
                home_team_pos_avrg['ypos'][jersey_number_t] += y_pos
                if x_pos!=0 or y_pos!=0:
                    home_team_pos_count[jersey_number_t] +=1
                
            elif has_ball_teamId_t == away_team_id:
                home_team_pos_avrg['xpos'][jersey_number_t] += x_pos
                home_team_pos_avrg['ypos'][jersey_number_t] += y_pos
                if x_pos!=0 or y_pos!=0:
                    home_team_pos_count[jersey_number_t] +=1     
                    

        elif team_id_t == away_team_id:
            
            if player_positionId['awayTeam'][jersey_number_t] == 1:
                away_team_pos_avrg['xpos'][jersey_number_t] += x_pos
                away_team_pos_avrg['ypos'][jersey_number_t] += y_pos
                if x_pos!=0 or y_pos!=0:
                    away_team_pos_count[jersey_number_t] +=1
            
            elif has_ball_teamId_t == home_team_id:
                away_team_pos_avrg['xpos'][jersey_number_t] += x_pos
                away_team_pos_avrg['ypos'][jersey_number_t] += y_pos
                if x_pos!=0 or y_pos!=0:
                    away_team_pos_count[jersey_number_t] +=1    

        elif team_id_t == 0:
            if jersey_number_t == referee_id:
                pass
                # referee_pos_t['xpos']
                # referee_pos_t['ypos']
            else:
                pass
                    
        
    



600
599
{'xpos': {1: 18.69570165341471, 2: 0, 5: 36.7774676079394, 6: 36.61368948043976, 7: 52.64182642632588, 8: 51.263806350800735, 9: 0, 10: 49.70823444586924, 11: 0, 13: 44.6262622021318, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 63.55707443867774, 29: 0, 32: 39.18956350986845, 70: 0, 88: 39.71934802165303, 94: 57.72845157281839}, 'ypos': {1: 34.56082653400416, 2: 0, 5: 34.58907042431674, 6: 22.55563703703704, 7: 46.78863265807238, 8: 11.541266268578985, 9: 0, 10: 27.914529670002622, 11: 0, 13: 30.200695697304464, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 28.136041869895998, 29: 0, 32: 47.39620674888851, 70: 0, 88: 12.436225377160177, 94: 29.729874840485888}}
{'xpos': {1: 0, 3: 76.708935749057, 5: 0, 7: 0, 8: 55.59071780877452, 9: 42.6370307679762, 11: 60.512517435986496, 14: 63.41864092197573, 17: 0, 18: 0, 20: 0, 22: 0, 23: 70.75203039020126, 24: 73.23621117442339, 25: 95.695214599908, 28: 73.9176347004528, 30: 66.86087550607476, 35: 0, 70: 0, 77: 0, 90: 57.48716554837886}, 'ypos': {



4200
4199
{'xpos': {1: 12.870355790389237, 2: 0, 5: 26.93004604811005, 6: 23.566042611683592, 7: 35.03327186593479, 8: 30.10087285223387, 9: 0, 10: 31.013037800687353, 11: 0, 13: 34.219880202747184, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 38.31614226804108, 29: 0, 32: 23.95476701030915, 70: 0, 88: 26.001591350456614, 94: 31.56655430711608}, 'ypos': {1: 32.46800586041866, 2: 0, 5: 32.54246048109949, 6: 25.722360137457116, 7: 17.416963568848068, 8: 21.41138831615108, 9: 0, 10: 24.70146391752584, 11: 0, 13: 24.782997636796086, 14: 0, 15: 0, 17: 0, 18: 0, 20: 0, 23: 27.607632989690735, 29: 0, 32: 36.7151484536079, 70: 0, 88: 15.307365358510086, 94: 24.841605243445688}}
{'xpos': {1: 0, 3: 65.30732242424257, 5: 0, 7: 62.09522424242417, 8: 57.191156363636495, 9: 40.65869333333336, 11: 53.046204848484834, 14: 48.748145670995655, 17: 52.39519515151526, 18: 0, 20: 0, 22: 0, 23: 66.7520923867546, 24: 70.0238000000001, 25: 89.14494684509418, 28: 67.79359999999993, 30: 0, 35: 0, 70: 0, 77: 0, 90: 